In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow

In [3]:
from tensorflow import keras

In [4]:
from tensorflow.keras.models import Sequential

In [5]:
from tensorflow.keras.layers import Dense,Embedding,Flatten,Dropout,Activation
from tensorflow.keras.activations import sigmoid,softmax,relu
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
data=pd.read_csv('spam.csv')
df=pd.DataFrame(data)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
df=df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [8]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
import nltk

In [10]:
from nltk.corpus import stopwords

In [11]:
from nltk.stem import PorterStemmer,WordNetLemmatizer

In [12]:
df.columns

Index(['v1', 'v2'], dtype='object')

In [13]:
a=['result','message']

In [14]:
df.columns=a

In [15]:
df.head()

,result,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
df.isnull().sum()

result     0
message    0
dtype: int64

In [17]:
df['message']

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will �_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [18]:
len(df['message'])

5572

In [19]:
ps=PorterStemmer()

In [20]:
lemm=WordNetLemmatizer()

In [21]:
import re

In [22]:
for i in range(len(df['message'])):
    review=re.sub('[^A-Za-z]',' ',df['message'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(j) for j in review if j not in stopwords.words('english')]
    df['message'][i]=' '.join(review)

In [23]:
df['message']

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri wkli comp win fa cup final tkt st m...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    nd time tri contact u u pound prize claim easi...
5568                                b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: message, Length: 5572, dtype: object

In [24]:
df['result']=df['result'].map({'ham':1,'spam':0}).astype(int)

In [25]:
df.head()

,result,message
0,1,go jurong point crazi avail bugi n great world...
1,1,ok lar joke wif u oni
2,0,free entri wkli comp win fa cup final tkt st m...
3,1,u dun say earli hor u c alreadi say
4,1,nah think goe usf live around though


In [26]:
from tensorflow.keras.preprocessing.text import one_hot

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
#ONE HOT

In [29]:
voc_size=1000
embed_message=[one_hot(i,voc_size) for i in df['message']]
print(embed_message)

[[313, 690, 474, 906, 526, 515, 33, 314, 832, 345, 247, 870, 564, 471, 539, 316], [258, 536, 57, 963, 156, 732], [871, 364, 816, 113, 302, 949, 734, 851, 604, 137, 683, 788, 949, 650, 364, 450, 158, 191, 973, 94, 306], [156, 221, 728, 967, 398, 156, 94, 572, 728], [785, 960, 912, 194, 425, 830, 663], [282, 997, 185, 824, 205, 581, 427, 862, 121, 723, 258, 929, 158, 909, 155, 369], [408, 568, 427, 497, 839, 427, 913, 79], [737, 842, 518, 518, 702, 640, 517, 101, 366, 190, 658, 290, 23, 517, 190], [113, 638, 535, 894, 388, 547, 188, 398, 471, 670, 471, 590, 114, 119, 285], [866, 359, 156, 508, 465, 967, 742, 275, 866, 485, 871, 670, 866, 967, 69, 871], [203, 792, 607, 627, 855, 518, 179, 422, 617, 962, 893, 310], [214, 427, 302, 751, 638, 191, 901, 155, 222, 351, 476, 476, 267, 306, 359, 734, 891], [671, 824, 871, 626, 188, 141, 191, 205, 471, 94, 357, 533, 458, 628, 963, 285, 862], [825, 538, 205, 827, 966, 653, 281, 740, 750, 992, 949, 653, 228, 824, 706], [873, 476], [547, 726, 51, 64

In [30]:
len(embed_message)

5572

In [31]:
a=[]
for j in range(len(embed_message)):
    q=len(embed_message[j])
    a.append(q)

In [32]:
max(a)

77

In [33]:
#pad_sequence

In [34]:
sent_length=77
embed_message_pad=pad_sequences(embed_message,maxlen=sent_length,padding='post')
print(embed_message_pad)

[[313 690 474 ...   0   0   0]
 [258 536  57 ...   0   0   0]
 [871 364 816 ...   0   0   0]
 ...
 [944 211  23 ...   0   0   0]
 [662  35 376 ...   0   0   0]
 [931 556 231 ...   0   0   0]]


In [35]:
label=df['result']

In [36]:
label

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: result, Length: 5572, dtype: int32

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout,Embedding,LSTM,Bidirectional
from tensorflow.keras.activations import sigmoid,relu

In [46]:
model=Sequential()
model.add(Embedding(voc_size,10,input_length=sent_length))
model.add(LSTM(50))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))

In [47]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [88]:
model.fit(embed_message_pad,label,epochs=40,verbose=2)

Epoch 1/40
175/175 - 6s - loss: 0.4196 - accuracy: 0.8659
Epoch 2/40
175/175 - 6s - loss: 0.3960 - accuracy: 0.8659
Epoch 3/40
175/175 - 6s - loss: 0.3954 - accuracy: 0.8659
Epoch 4/40
175/175 - 6s - loss: 0.3948 - accuracy: 0.8659
Epoch 5/40
175/175 - 6s - loss: 0.3946 - accuracy: 0.8659
Epoch 6/40
175/175 - 6s - loss: 0.3945 - accuracy: 0.8659
Epoch 7/40
175/175 - 6s - loss: 0.3948 - accuracy: 0.8659
Epoch 8/40
175/175 - 6s - loss: 0.3945 - accuracy: 0.8659
Epoch 9/40
175/175 - 6s - loss: 0.3952 - accuracy: 0.8659
Epoch 10/40
175/175 - 7s - loss: 0.4142 - accuracy: 0.8487
Epoch 11/40
175/175 - 7s - loss: 0.3939 - accuracy: 0.8659
Epoch 12/40
175/175 - 6s - loss: 0.3943 - accuracy: 0.8659
Epoch 13/40
175/175 - 7s - loss: 0.3932 - accuracy: 0.8659
Epoch 14/40
175/175 - 6s - loss: 0.3936 - accuracy: 0.8659
Epoch 15/40
175/175 - 5s - loss: 0.3925 - accuracy: 0.8659
Epoch 16/40
175/175 - 5s - loss: 0.3918 - accuracy: 0.8659
Epoch 17/40
175/175 - 6s - loss: 0.3927 - accuracy: 0.8659
Epoch 

In [66]:
df.head(15)

,result,message
0,1,go jurong point crazi avail bugi n great world...
1,1,ok lar joke wif u oni
2,0,free entri wkli comp win fa cup final tkt st m...
3,1,u dun say earli hor u c alreadi say
4,1,nah think goe usf live around though
5,0,freemsg hey darl week word back like fun still...
6,1,even brother like speak treat like aid patent
7,1,per request mell mell oru minnaminungint nurun...
8,0,winner valu network custom select receivea pri...
9,0,mobil month u r entitl updat latest colour mob...


In [38]:
import sklearn

In [39]:
from sklearn.model_selection import train_test_split

In [50]:
X

array(['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
       'ok lar joke wif u oni',
       'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
       ..., 'piti mood suggest',
       'guy bitch act like interest buy someth els next week gave us free',
       'rofl true name'], dtype=object)

In [51]:
X=np.array(embed_message_pad)
y=np.array(df['result'])

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [60]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),verbose = 2 , batch_size = 64 , epochs = 20)

Epoch 1/20
70/70 - 4s - loss: 0.4503 - accuracy: 0.8571 - val_loss: 0.3934 - val_accuracy: 0.8655
Epoch 2/20
70/70 - 3s - loss: 0.3408 - accuracy: 0.8815 - val_loss: 0.2780 - val_accuracy: 0.9453
Epoch 3/20
70/70 - 3s - loss: 0.2368 - accuracy: 0.8645 - val_loss: 0.1930 - val_accuracy: 0.9354
Epoch 4/20
70/70 - 3s - loss: 0.1634 - accuracy: 0.9506 - val_loss: 0.2183 - val_accuracy: 0.9462
Epoch 5/20
70/70 - 5s - loss: 0.1695 - accuracy: 0.9585 - val_loss: 0.2162 - val_accuracy: 0.9435
Epoch 6/20
70/70 - 5s - loss: 0.1670 - accuracy: 0.9592 - val_loss: 0.2174 - val_accuracy: 0.9435
Epoch 7/20
70/70 - 5s - loss: 0.1658 - accuracy: 0.9596 - val_loss: 0.2190 - val_accuracy: 0.9435
Epoch 8/20
70/70 - 5s - loss: 0.1634 - accuracy: 0.9605 - val_loss: 0.2177 - val_accuracy: 0.9435
Epoch 9/20
70/70 - 4s - loss: 0.1620 - accuracy: 0.9610 - val_loss: 0.2208 - val_accuracy: 0.9435
Epoch 10/20
70/70 - 4s - loss: 0.1618 - accuracy: 0.9612 - val_loss: 0.2222 - val_accuracy: 0.9435
Epoch 11/20
70/70 -

In [61]:
y_train_pred=model.predict_classes(X_train)

In [62]:
y_test_pred=model.predict_classes(X_test)

In [63]:
y_train_pred[:15]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [64]:
accuracy_score(y_test,y_test_pred)

0.9408071748878923

In [65]:
accuracy_score(y_train,y_train_pred)

0.9634283150100965